In [1]:
import pandas as pd
import numpy as np
from utils import encode_data_get_embeddings, create_emoji_sentiment
import model as models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense
from sklearn.metrics import f1_score, classification_report
import tensorflow as tf

## Nacitanie dat a embedingov

In [2]:
# tweets = pd.read_csv('D:/Downloads/DP/data/e2v_data.csv')
tweets = pd.read_csv('D:/Downloads/DP/data/e2v_data_emoji.csv')

# tweets = pd.read_csv('D:/Downloads/DP/data/emocontext.csv')
# tweets = pd.read_csv('D:/Downloads/DP/data/emocontext_emoji.csv')
# tweets_dev = pd.read_csv('D:/Downloads/DP/data/emocontext_dev.csv')
# tweets_test = pd.read_csv('D:/Downloads/DP/data/emocontext_test.csv')
# tw_cnt = len(tweets)
# tweets = pd.concat([tweets,tweets_test])

In [3]:
w2v = open('D:\Downloads\DP\word2vec300_en.vec', encoding="utf8")
w2v_dim = w2v.readline()
print(w2v_dim)

e2v = open('D:\Downloads\DP\emoji2vec300.txt', encoding="utf8")
e2v_dim = e2v.readline()
print(e2v_dim)

train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, None)
# train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, tw_cnt, w2v, e2v)

2000000 300

1661 300



## Predikcie - Neuronove siete

### BiLstm - emosent data

In [14]:
classes = ['others', 'sad', 'happy', 'angry']
predict = model.predict(test_x)
predict = np.argmax(predict, axis=1)
predict = [classes[pr] for pr in predict]

In [43]:
model = Sequential()
model.add(Embedding(embedding_matrix.shape[0],
                 300,
                 weights=[embedding_matrix],
                 input_length=80,
                 trainable=False))

model.add(Bidirectional(LSTM(units=256,
            dropout=0.1,
            recurrent_dropout=0.1
                            )))

model.add(Dense(1, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 81, 300)           3668400   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               1140736   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 513       
Total params: 4,809,649
Trainable params: 1,141,249
Non-trainable params: 3,668,400
_________________________________________________________________


In [ ]:
model.fit(train_x, train_y, 
          batch_size=64,
          epochs=10,
          validation_split=0.2)

In [15]:
print(f1_score(tweets.Label[tw_cnt:], predict, average='micro', labels=['sad', 'happy', 'angry']))
print(classification_report(tweets.Label[tw_cnt:], predict))


0.5729836714497774
              precision    recall  f1-score   support

       angry       0.54      0.80      0.65       298
       happy       0.39      0.63      0.48       284
      others       0.95      0.88      0.91      4677
         sad       0.55      0.65      0.59       250

    accuracy                           0.85      5509
   macro avg       0.61      0.74      0.66      5509
weighted avg       0.88      0.85      0.86      5509



### BiLstm - e2v data


In [2]:
tweets = pd.read_csv('D:/Downloads/DP/data/e2v_data_emoji.csv')

w2v = open('D:\Downloads\DP\word2vec300_en.vec', encoding="utf8")
w2v_dim = w2v.readline()
print(w2v_dim)

e2v = open('D:\Downloads\DP\emoji2vec300.txt', encoding="utf8")
e2v_dim = e2v.readline()
print(e2v_dim)

train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, None)
# train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, e2v)

2000000 300

1661 300



In [3]:
model = models.BiLstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=64,
    validation_split=0.2)

Epoch 1/10
109/109 [==============================] - 60s 546ms/step - loss: 0.9366 - acc: 0.5681 - val_loss: 0.9052 - val_acc: 0.5865
Epoch 2/10
109/109 [==============================] - 57s 522ms/step - loss: 0.8622 - acc: 0.5992 - val_loss: 0.8759 - val_acc: 0.5911
Epoch 3/10
109/109 [==============================] - 57s 521ms/step - loss: 0.8227 - acc: 0.6194 - val_loss: 0.8667 - val_acc: 0.5871
Epoch 4/10
109/109 [==============================] - 57s 521ms/step - loss: 0.7984 - acc: 0.6294 - val_loss: 0.8746 - val_acc: 0.5929
Epoch 5/10
109/109 [==============================] - 57s 526ms/step - loss: 0.7805 - acc: 0.6471 - val_loss: 0.8503 - val_acc: 0.5975
Epoch 6/10
109/109 [==============================] - 84s 773ms/step - loss: 0.7622 - acc: 0.6544 - val_loss: 0.8539 - val_acc: 0.6009
Epoch 7/10
109/109 [==============================] - 86s 787ms/step - loss: 0.7399 - acc: 0.6635 - val_loss: 0.8677 - val_acc: 0.6124
Epoch 8/10
109/109 [==============================] - 8

In [4]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: 1.0444302558898926, acc: 0.5089737772941589


In [5]:
tweets = pd.read_csv('D:/Downloads/DP/data/e2v_data_emoji.csv')

w2v = open('D:\Downloads\DP\word2vec300_en.vec', encoding="utf8")
w2v_dim = w2v.readline()
print(w2v_dim)

e2v = open('D:\Downloads\DP\emoji2vec300.txt', encoding="utf8")
e2v_dim = e2v.readline()
print(e2v_dim)

# train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, None)
train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, e2v)

2000000 300

1661 300



In [6]:
model = models.BiLstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=64,
    validation_split=0.2)

Epoch 1/10
109/109 [==============================] - 61s 558ms/step - loss: 0.8700 - acc: 0.5972 - val_loss: 0.8776 - val_acc: 0.6113
Epoch 2/10
109/109 [==============================] - 63s 576ms/step - loss: 0.8078 - acc: 0.6407 - val_loss: 0.8673 - val_acc: 0.6107
Epoch 3/10
109/109 [==============================] - 64s 585ms/step - loss: 0.7755 - acc: 0.6494 - val_loss: 0.8542 - val_acc: 0.6182
Epoch 4/10
109/109 [==============================] - 63s 578ms/step - loss: 0.7425 - acc: 0.6689 - val_loss: 0.8517 - val_acc: 0.6055
Epoch 5/10
109/109 [==============================] - 63s 579ms/step - loss: 0.7100 - acc: 0.6886 - val_loss: 0.8691 - val_acc: 0.6164
Epoch 6/10
109/109 [==============================] - 64s 583ms/step - loss: 0.6741 - acc: 0.6991 - val_loss: 0.8722 - val_acc: 0.6216
Epoch 7/10
109/109 [==============================] - 84s 770ms/step - loss: 0.6452 - acc: 0.7229 - val_loss: 0.9217 - val_acc: 0.6049
Epoch 8/10
109/109 [==============================] - 9

In [7]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: 0.9752941727638245, acc: 0.6000920534133911


In [8]:
tweets = pd.read_csv('D:/Downloads/DP/data/e2v_data.csv')

w2v = open('D:\Downloads\DP\word2vec300_en.vec', encoding="utf8")
w2v_dim = w2v.readline()
print(w2v_dim)

e2v = open('D:\Downloads\DP\emoji2vec300.txt', encoding="utf8")
e2v_dim = e2v.readline()
print(e2v_dim)

train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, None)
# train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, e2v)

2000000 300

1661 300



In [9]:
model = models.BiLstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=64,
    validation_split=0.2)

Epoch 1/10
646/646 [==============================] - 381s 590ms/step - loss: 0.8908 - acc: 0.5858 - val_loss: 0.8307 - val_acc: 0.6277
Epoch 2/10
646/646 [==============================] - 397s 614ms/step - loss: 0.8177 - acc: 0.6306 - val_loss: 0.8071 - val_acc: 0.6319
Epoch 3/10
646/646 [==============================] - 396s 613ms/step - loss: 0.7895 - acc: 0.6466 - val_loss: 0.7871 - val_acc: 0.6502
Epoch 4/10
646/646 [==============================] - 397s 615ms/step - loss: 0.7725 - acc: 0.6556 - val_loss: 0.7838 - val_acc: 0.6481
Epoch 5/10
646/646 [==============================] - 437s 677ms/step - loss: 0.7561 - acc: 0.6640 - val_loss: 0.7766 - val_acc: 0.6553
Epoch 6/10
646/646 [==============================] - 394s 610ms/step - loss: 0.7404 - acc: 0.6713 - val_loss: 0.7789 - val_acc: 0.6529
Epoch 7/10
646/646 [==============================] - 390s 604ms/step - loss: 0.7226 - acc: 0.6811 - val_loss: 0.7774 - val_acc: 0.6573
Epoch 8/10
646/646 [============================

In [10]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: 0.8660539388656616, acc: 0.6269350051879883


In [2]:
tweets = pd.read_csv('D:/Downloads/DP/data/e2v_data.csv')

w2v = open('D:\Downloads\DP\word2vec300_en.vec', encoding="utf8")
w2v_dim = w2v.readline()
print(w2v_dim)

e2v = open('D:\Downloads\DP\emoji2vec300.txt', encoding="utf8")
e2v_dim = e2v.readline()
print(e2v_dim)

# train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, None)
train_x, train_y, test_x, test_y, embedding_matrix = encode_data_get_embeddings(tweets, 0.8, w2v, e2v)

2000000 300

1661 300



In [3]:
model = models.BiLstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=64,
    validation_split=0.2)

Epoch 1/10
646/646 [==============================] - 381s 589ms/step - loss: 0.8845 - acc: 0.5900 - val_loss: 0.8399 - val_acc: 0.6203
Epoch 2/10
646/646 [==============================] - 352s 545ms/step - loss: 0.8115 - acc: 0.6377 - val_loss: 0.8081 - val_acc: 0.6416
Epoch 3/10
646/646 [==============================] - 401s 621ms/step - loss: 0.7826 - acc: 0.6514 - val_loss: 0.7883 - val_acc: 0.6507
Epoch 4/10
646/646 [==============================] - 343s 531ms/step - loss: 0.7618 - acc: 0.6623 - val_loss: 0.7823 - val_acc: 0.6554
Epoch 5/10
646/646 [==============================] - 354s 547ms/step - loss: 0.7452 - acc: 0.6706 - val_loss: 0.7724 - val_acc: 0.6611
Epoch 6/10
646/646 [==============================] - 397s 614ms/step - loss: 0.7262 - acc: 0.6793 - val_loss: 0.7797 - val_acc: 0.6578
Epoch 7/10
646/646 [==============================] - 341s 527ms/step - loss: 0.7069 - acc: 0.6902 - val_loss: 0.7801 - val_acc: 0.6564
Epoch 8/10
646/646 [============================

In [4]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: 0.851948618888855, acc: 0.6384674906730652


## BiLstm model

In [9]:
model = models.BiLstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=64,
    validation_split=0.2)

Epoch 1/10
302/302 [==============================] - 117s 389ms/step - loss: 0.8910 - acc: 0.6539 - val_loss: 0.6864 - val_acc: 0.7261
Epoch 2/10
302/302 [==============================] - 104s 343ms/step - loss: 0.5986 - acc: 0.7704 - val_loss: 0.5821 - val_acc: 0.7793
Epoch 3/10
302/302 [==============================] - 105s 348ms/step - loss: 0.5073 - acc: 0.8099 - val_loss: 0.5337 - val_acc: 0.7984
Epoch 4/10
302/302 [==============================] - 105s 348ms/step - loss: 0.4685 - acc: 0.8220 - val_loss: 0.5300 - val_acc: 0.8040
Epoch 5/10
302/302 [==============================] - 105s 349ms/step - loss: 0.4275 - acc: 0.8405 - val_loss: 0.4784 - val_acc: 0.8257
Epoch 6/10
302/302 [==============================] - 106s 350ms/step - loss: 0.3968 - acc: 0.8557 - val_loss: 0.4467 - val_acc: 0.8361
Epoch 7/10
302/302 [==============================] - 106s 350ms/step - loss: 0.3733 - acc: 0.8645 - val_loss: 0.4531 - val_acc: 0.8373
Epoch 8/10
302/302 [============================

In [10]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: 0.430917888879776, acc: 0.8461538553237915


## Testing model

In [ ]:
model = models.TestModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=32,
    validation_split=0.2)

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

### Combi model

In [10]:
model = models.BiLstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=32,
    validation_split=0.2)

Epoch 1/10
1292/1292 [==============================] - 554s 429ms/step - loss: 0.0798 - acc: 0.4610 - val_loss: 0.5632 - val_acc: 0.4553
Epoch 2/10
1292/1292 [==============================] - 545s 421ms/step - loss: -0.2002 - acc: 0.4619 - val_loss: 0.0506 - val_acc: 0.4574
Epoch 3/10
1292/1292 [==============================] - 564s 437ms/step - loss: -0.0424 - acc: 0.4650 - val_loss: -0.2589 - val_acc: 0.4577
Epoch 4/10
1292/1292 [==============================] - 583s 451ms/step - loss: -0.2362 - acc: 0.4639 - val_loss: -0.1299 - val_acc: 0.4602
Epoch 5/10
1292/1292 [==============================] - 572s 443ms/step - loss: -0.8435 - acc: 0.4725 - val_loss: -1.1468 - val_acc: 0.5018
Epoch 6/10
1292/1292 [==============================] - 569s 440ms/step - loss: -1.1712 - acc: 0.4839 - val_loss: -1.3310 - val_acc: 0.4689
Epoch 7/10
1292/1292 [==============================] - 540s 418ms/step - loss: -0.8657 - acc: 0.4749 - val_loss: -1.2684 - val_acc: 0.4702
Epoch 8/10
1292/1292 [=

In [11]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: -1.3393548727035522, acc: 0.5148705244064331


### Conv model

In [12]:
model = models.ConvModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=5,
    batch_size=32,
    validation_split=0.2)

Epoch 1/5
1292/1292 [==============================] - 89s 69ms/step - loss: -988483.5000 - acc: 0.4483 - val_loss: -4403449.0000 - val_acc: 0.4309
Epoch 2/5
1292/1292 [==============================] - 87s 67ms/step - loss: -27949140.0000 - acc: 0.4437 - val_loss: -57549448.0000 - val_acc: 0.4421
Epoch 3/5
1292/1292 [==============================] - 87s 67ms/step - loss: -149946816.0000 - acc: 0.4436 - val_loss: -222597328.0000 - val_acc: 0.4482
Epoch 4/5
1292/1292 [==============================] - 95s 74ms/step - loss: -460484768.0000 - acc: 0.4442 - val_loss: -571984832.0000 - val_acc: 0.4502
Epoch 5/5
1292/1292 [==============================] - 105s 81ms/step - loss: -1016310080.0000 - acc: 0.4449 - val_loss: -1172016640.0000 - val_acc: 0.4497


In [13]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: -1159315328.0, acc: 0.45410215854644775


### Baseline model
- e2v data: 46% (with/out e2v)
- e2v_emoji data: 25% (with/out e2v)

In [6]:
model = models.BaselineModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=5,
    batch_size=32,
    validation_split=0.2)

Epoch 1/5
3679/3679 [==============================] - 87s 24ms/step - loss: 0.1552 - acc: 0.1649 - val_loss: 0.1269 - val_acc: 0.1637
Epoch 2/5
3679/3679 [==============================] - 83s 23ms/step - loss: 0.1453 - acc: 0.1624 - val_loss: 0.1179 - val_acc: 0.1631
Epoch 3/5
3679/3679 [==============================] - 83s 23ms/step - loss: 0.1310 - acc: 0.1613 - val_loss: 0.1196 - val_acc: 0.1630
Epoch 4/5
3679/3679 [==============================] - 83s 23ms/step - loss: 0.1287 - acc: 0.1612 - val_loss: 0.1182 - val_acc: 0.1630
Epoch 5/5
3679/3679 [==============================] - 83s 23ms/step - loss: 0.1291 - acc: 0.1612 - val_loss: 0.1175 - val_acc: 0.1630


In [7]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: 0.10143047571182251, acc: 0.1616787612438202


### BaselineDrop model

In [6]:
model = models.BaselineDropModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=5,
    batch_size=32,
    validation_split=0.2)

Epoch 1/5
1292/1292 [==============================] - 35s 27ms/step - loss: 0.1600 - acc: 0.4608 - val_loss: 0.1721 - val_acc: 0.4550
Epoch 2/5
1292/1292 [==============================] - 35s 27ms/step - loss: 0.1503 - acc: 0.4612 - val_loss: 0.1794 - val_acc: 0.4550
Epoch 3/5
1292/1292 [==============================] - 36s 28ms/step - loss: 0.1530 - acc: 0.4610 - val_loss: 0.1714 - val_acc: 0.4569
Epoch 4/5
1292/1292 [==============================] - 40s 31ms/step - loss: 0.1960 - acc: 0.4572 - val_loss: 0.3097 - val_acc: 0.4471
Epoch 5/5
1292/1292 [==============================] - 41s 31ms/step - loss: 0.2088 - acc: 0.4567 - val_loss: 0.1994 - val_acc: 0.4549


In [7]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: 0.18892277777194977, acc: 0.46122291684150696


### Lstm model

In [53]:
model = models.LstmModel(embedding_matrix)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics = ['acc'])  

model.fit(
    x=train_x,
    y=train_y,
    epochs=10,
    batch_size=64,
    validation_split=0.2)

Epoch 1/10
109/109 [==============================] - 13s 115ms/step - loss: -1.0244 - acc: 0.3762 - val_loss: -1.5107 - val_acc: 0.3341
Epoch 2/10
109/109 [==============================] - 11s 98ms/step - loss: -1.4884 - acc: 0.4302 - val_loss: -1.5820 - val_acc: 0.4598
Epoch 3/10
109/109 [==============================] - 11s 100ms/step - loss: -1.6164 - acc: 0.3240 - val_loss: -1.6535 - val_acc: 0.3059
Epoch 4/10
109/109 [==============================] - 10s 95ms/step - loss: -1.2929 - acc: 0.3465 - val_loss: -1.1082 - val_acc: 0.4153
Epoch 5/10
109/109 [==============================] - 11s 100ms/step - loss: -1.4010 - acc: 0.3545 - val_loss: -1.5625 - val_acc: 0.2516
Epoch 6/10
109/109 [==============================] - 11s 100ms/step - loss: -1.2555 - acc: 0.2757 - val_loss: -1.3604 - val_acc: 0.2708
Epoch 7/10
109/109 [==============================] - 11s 103ms/step - loss: -1.3701 - acc: 0.2894 - val_loss: -1.6187 - val_acc: 0.2590
Epoch 8/10
109/109 [=======================

In [54]:
score = model.evaluate(test_x, test_y, verbose=0)
print(f'loss: {score[0]}, acc: {score[1]}')

loss: -1.4610261917114258, acc: 0.2863207757472992


### Testing

In [66]:
embedding_matrix.shape

(101147, 300)

In [73]:
embedding_matrix.shape

(12228, 300)

In [10]:
import re
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import TweetTokenizer

tw_tok = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

x = np.array((tweets['Text']).apply(lambda x: ' '.join([w for w in tw_tok.tokenize(x)])))

# One-hot encoding labelu
y = np.array(tweets['Label'].apply(lambda x: 2 if x == 'Positive' else (1 if x =='Neutral' else 0)))
#     y = np.array(tweets['Label'].apply(lambda x: 1 if x == 'Positive' else 0))

tweet_count = len(tweets)
limit = int(0.8 * tweet_count)

train_x = x[:limit]
train_y = y[:limit]
test_x = x[limit:]
test_y = y[limit:]

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_x)

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

vocab_size = len(tokenizer.word_index) + 1

In [11]:
vocab_size

59945

In [12]:
embedding_matrix.shape[0]

59991